## El objetivo de este ejercicio es implementar q-learning y sarsa con aproximación de función de valor utilizando experience replay.

## Experience replay consiste en lo siguiente:

<img src="experience_replay.PNG">

## Recordar los targets de sarsa y q-learning respectivamente:

## Sarsa:
<img src="sarsa.PNG">

## Q-Learning:

<img src="q-learning.PNG">


In [ ]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import sklearn.preprocessing
from lib import plotting

ENV_NAME = "CartPole-v1"
env = gym.make(ENV_NAME)

In [ ]:
class Estimator():
    """
    Red Neuronal que aproxima la función de valor estado acción (q-function)
    """   
    def __init__(self, dimS, nA, learning_rate=0.001):

        self.model = Sequential()
        self.model.add(Dense(24, input_shape=(dimS,), activation="relu"))
        self.model.add(Dense(24, activation="relu"))
        self.model.add(Dense(nA, activation="linear"))
        self.model.compile(loss="mse", optimizer=Adam(lr=learning_rate))
     
    def update(self, states, q_values, verbose=0):
        """
        Realiza un update de los parámetros de la red neuronal usando un batch de estados y batch de vectores de valores
        de la función estado-acción correspondientes
        """
        self.model.fit(states, q_values, verbose=0)
    
    def predict(self, s):
        """
        Realiza una predicción de la función de valor estado-accion dado el estado
        
        Argumentos:
            s: estado para el cual realizar la predicción
            
        Retorna:
            Un vector con la predicción de la función de valor para todas las accións
        """
        return self.model.predict(s)


In [ ]:
class Learner:
    def __init__(self, env, estimator, num_episodes, discount_factor=1.0, 
                 exploration_max=1.0, exploration_min=0.01, epsilon_decay=0.99, memory_size=1000000, 
                 batch_size=20, run_online=False, use_qlearning=True):
        """
        Algoritmo q-learning/sarsa con experience replay utilizando aproximación de funciones.

        Argumentos de inicialización:
            env: ambiente de OpenAI.
            estimator: función de aproximación de la función de valor estado-acción.
            num_episodes: número de episodios durante los cuales entrenar el estimador.
            discount_factor: factor de descuento gama.
            exploration_max: probabilidad de tomar una acción aleatoria inicial.
            exploration_min: mínima probabilidad de tomar una acción aleatoria.
            epsilon_decay: en cada episodio la probabilidad de una acción aleatoria deace por este factor.
        """
        self.exploration_rate = exploration_max
        self.exploration_min = exploration_min
        self.env = env
        self.estimator = estimator
        self.num_episodes = num_episodes
        self.discount_factor = discount_factor
        self.epsilon_decay = epsilon_decay
        self.memory = deque(maxlen=memory_size)
        self.batch_size = batch_size
        self.run_online = run_online
        self.use_qlearning = use_qlearning
        
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def decay_exploration_rate(self):
        self.exploration_rate *= self.epsilon_decay
        self.exploration_rate = max(self.exploration_min, self.exploration_rate)
        
    def policy_fn(self, state, epsilon):
        """
        Política epsilon-greedy basada en la función de aproximación actual y la probabilidad de exploración epsilon.

        Retorna:
            Un vector con la probabilidad de tomar cada acción.
        """
        A = np.ones(env.action_space.n, dtype=float) * self.exploration_rate / env.action_space.n
        q_values = self.estimator.predict(state)
        best_action = np.argmax(q_values[0])
        A[best_action] += (1.0 - epsilon)
        return A
    
    def experience_replay(self):
        """
        Realiza una corrida de entrenamiento de la función de aproximación dado un batch de experiencia acumulada.
        """
        if len(self.memory) < self.batch_size:
            return
        
        batch = random.sample(self.memory, self.batch_size)
        
        # contienen un batch de 'experiencia' sampleada de la memoria acumulada
        batch_q_values = np.zeros((self.batch_size, self.env.action_space.n))
        batch_states = np.zeros((self.batch_size, self.env.observation_space.shape[0]))
        
        for i_batch, (state, action, reward, next_state, terminal) in enumerate(batch):
            
            q_update = reward
            if not terminal:
                ## q-learning:
                if self.use_qlearning:
                    # COMPLETAR!
                    # escribir el target de q-learning
                    #q_update =
                else: 
                ## sarsa:
                    # COMPLETAR!
                    # obtener las probabilidades de las acciones dado la política epsilon-greedy con
                    # respecto a la función de valor estado-acción actual y el próximo estado
                    # next_action_probs = 
                    # elegir la próxima acción aleatoriamente según esa distribución
                    # next_action =
                    # obtener los valores de la función de valor estado-acción para el próximo estado
                    # q_values_next = 
                    # escribir el target de sarsa para la acción seleccionada
                    # q_update = 
            
            # COMPLETAR
            # valores actuales de la función de valor para este estado:
            # q_values = 
            # substituir el valor de la acción actual por el target
            q_values[0][action] = q_update
            
            # llenar el array del batch
            batch_q_values[i_batch] = q_values
            batch_states[i_batch] = state
            i_batch += 1
        # avanzar en esa la dirección del gradiente dado los estados y los targets en el batch
        self.estimator.update(batch_states, batch_q_values, verbose=0)
        
    def train(self):    
        """
        Realiza Q-Learning o Sarsa con aproximación de la función de valor estado-acción.
        Retorna:
            Un objeto de tipo EpisodeStats con dos arrays de numpy para la longitud y recompensa acumulada de cada
            episodio respectivamente.
        """
        stats = plotting.EpisodeStats(
            episode_lengths=np.zeros(self.num_episodes),
            episode_rewards=np.zeros(self.num_episodes))    
        
        for run in range(self.num_episodes):
            state = env.reset()
            state = np.reshape(state, [1, self.env.observation_space.shape[0]])
            step = 0
            cum_reward = 0.0
            
            action = np.random.choice(self.env.action_space.n, p=self.policy_fn(state, self.exploration_rate))
            
            while True:
                env.render()
                
                next_state, reward, terminal, info = env.step(action)
                
                reward = reward if not terminal else -reward
                cum_reward += reward*self.discount_factor**step
                
                next_state = np.reshape(next_state, [1, self.env.observation_space.shape[0]])
                
                if not terminal:
                    next_action = np.random.choice(self.env.action_space.n, p=self.policy_fn(next_state, self.exploration_rate))
                else:
                    next_action = None
                
                
                self.remember(state, action, reward, next_state, terminal)
                
                if self.run_online:
                    # este es el caso en que hacemos un update en cada paso de experiencia:
                    q_update = reward
                    if not terminal:
                        ## q-learning:
                        if self.use_qlearning:
                            # COMPLETAR!
                            # escribir el target de q-learning
                            # q_update = 
                        else: 
                        ## sarsa:
                        # COMPLETAR!
                            # obtener las probabilidades de las acciones dado la política epsilon-greedy con
                            # respecto a la función de valor estado-acción actual y el próximo estado
                            # next_action_probs = 
                            # elegir la próxima acción aleatoriamente según esa distribución
                            # next_action =
                            # obtener los valores de la función de valor estado-acción para el próximo estado
                            # q_values_next = 
                            # escribir el target de sarsa para la acción seleccionada
                            # q_update = 
                    # COMPLETAR!
                    # valores actuales de la función de valor para este estado:
                    # q_values = 
                    q_values[0][action] = q_update
                    # hacer update en la dirección del gradiente
                    self.estimator.update([state], [q_values], verbose=0)
                    
                
                state = next_state
                action = next_action
                
                step += 1
                if terminal:
                    print("Episodio: " + str(run) + ", Exploración: " + str(self.exploration_rate) + 
                          ", Recompensa Acumulada: " + str(cum_reward))
                    # Actualizar estadísticas
                    stats.episode_rewards[run] = cum_reward
                    stats.episode_lengths[run] = step
                    break        
                    
                if not self.run_online:
                    self.experience_replay()
            self.decay_exploration_rate()
        return stats

In [ ]:
estimator = Estimator(env.observation_space.shape[0], env.action_space.n)
learner = Learner(env, estimator, 150, run_online=False, use_qlearning=True)

In [ ]:
stats = learner.train()

In [ ]:
plotting.plot_episode_stats(stats, smoothing_window=25)